In [30]:
from pickle import dump 
import re
import numpy as np
from unicodedata import normalize
import string

from pickle import load
from numpy.random import rand
from numpy.random import shuffle

<h3>Read data</h3>
<p>Data from http://www.manythings.org/, German words and their translations</p>

In [17]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [18]:
doc = load_doc('data/deu-eng/deu.txt')

In [19]:
doc[:20]

'Hi.\tHallo!\nHi.\tGrüß '

<h3>Cleaning the data</h3>

In [20]:
def to_pairs(doc):
    lines = doc.split('\n')
    pairs = [x.split('\t') for x in lines]
    return pairs

In [21]:
pairs = to_pairs(doc)
pairs[:5]

[['Hi.', 'Hallo!'],
 ['Hi.', 'Grüß Gott!'],
 ['Run!', 'Lauf!'],
 ['Fire!', 'Feuer!'],
 ['Help!', 'Hilfe!']]


- Remove all non-printable characters.
- Remove all punctuation characters.
- Normalize all Unicode characters to ASCII (e.g. Latin characters).
- Normalize the case to lowercase.
- Remove any remaining tokens that are not alphabetic.


In [26]:
# clean a list of lines
def clean_pairs(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return np.array(cleaned)

The unicodedata module offers a .normalize() function, you want to normalize to the NFC form. NFC, or 'Normal Form Composed' returns composed characters, NFD, 'Normal Form Decomposed' gives you decomposed, combined characters.

In [27]:
cleaned_pairs = clean_pairs(pairs)
cleaned_pairs[:5]

array([list(['hi', 'hallo']), list(['hi', 'gru gott']),
       list(['run', 'lauf']), list(['fire', 'feuer']),
       list(['help', 'hilfe'])], dtype=object)

<h3>Save the cleansed data</h3>

In [32]:
def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)

In [38]:
save_clean_data(cleaned_pairs, "data/deu-eng/cleansed_deu.pkl")

Saved: data/deu-eng/cleansed_deu.pkl


<h3>Loading the processed data</h3>

In [39]:
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))

In [40]:
raw_dataset = load_clean_sentences('data/deu-eng/cleansed_deu.pkl')

In [41]:
raw_dataset[:5]

array([list(['hi', 'hallo']), list(['hi', 'gru gott']),
       list(['run', 'lauf']), list(['fire', 'feuer']),
       list(['help', 'hilfe'])], dtype=object)

In [42]:
len(raw_dataset)

169814

<h3>Test Train Split</h3>

In [44]:
n_sentences = 10000
dataset = raw_dataset[:n_sentences]
# random shuffle
shuffle(dataset)
# split into train/test
train, test = dataset[:9000], dataset[9000:]

In [45]:
train[:5], len(train)

(array([list(['do it tomorrow', 'machs morgen']),
        list(['i can do more', 'ich kann mehr machen']),
        list(['tom applauded', 'tom hat geklatscht']),
        list(['who found you', 'wer hat dich gefunden']),
        list(['look up there', 'sieh dort oben nach'])], dtype=object), 9000)

In [46]:
test[:5], len(test)

(array([list(['i teach spanish', 'ich unterrichte spanisch']),
        list(['shes too loud', 'sie ist zu laut']),
        list(['start now', 'fang jetzt an']),
        list(['lying is wrong', 'es ist nicht recht zu lugen']),
        list(['they hired tom', 'tom wurde eingestellt'])], dtype=object),
 1000)

<h3>Tokenize</h3>

In [48]:
dataset[:5, 0]

IndexError: too many indices for array